<a href="https://colab.research.google.com/github/achyutpratap6/Medical-Image-classification-using-Deep-Learning/blob/main/dab322_capstone_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/grive')

Mounted at /content/grive


In [ ]:
!nvidia-smi

Wed Jul 26 05:38:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns

from imutils import paths
import argparse
import random
import pickle
import cv2
import os

In [ ]:
# Path of Train data folder
data_dir = '/content/grive/MyDrive/Data/Full'
# Image Size
IMAGE_SIZE = (150,150)

In [ ]:
# Initialize the data and labels as an empty list
# Reshape the image data and append it in the list-data
data = []
# Encode the image labels and append it in the list-labels
labels = []

# Create single list by of absolute path of images by concatenating the file name of each images with its respective folder name.
TraImagePaths = sorted(list(paths.list_images(data_dir)))
# Total number images in Training
total_number_of_images = len(TraImagePaths)
print("Total number of Images----->",total_number_of_images)


Total number of Images-----> 300


In [ ]:
# Randomly shuffle all the image file name
random.shuffle(TraImagePaths)

In [ ]:
# Check
TraImagePaths

['/content/grive/MyDrive/Data/Full/Bed/Baxton Studio Adela Modern and Contemporary Grey Finished Wood Queen Size Platform Bed.jpg',
 '/content/grive/MyDrive/Data/Full/Bed/Anya Full Fabric Bed.jpg',
 '/content/grive/MyDrive/Data/Full/Sofa/Allure Upholstered Sofa.jpg',
 '/content/grive/MyDrive/Data/Full/Sofa/Baxton Studio Ally Modern and Contemporary Charcoal Fabric Upholstered Twin Size Sofa Daybed with Roll Out Trundle Guest Bed.jpg',
 '/content/grive/MyDrive/Data/Full/Bed/Amelia Twin Performance Velvet Bed.jpg',
 '/content/grive/MyDrive/Data/Full/Sofa/3 Seater Sofa.jpg',
 '/content/grive/MyDrive/Data/Full/Sofa/Armen Living Everly Contemporary Sofa in Genuine Black Leather with Brushed Stainless Steel Legs.jpg',
 '/content/grive/MyDrive/Data/Full/Sofa/Baxton Studio Amaris Modern and Contemporary Grey Bonded Leather 5-Piece Power Reclining Sectional Sofa with USB Ports.jpg',
 '/content/grive/MyDrive/Data/Full/Chair/Ardor Office Chair in Black .jpg',
 '/content/grive/MyDrive/Data/Full/Be

In [ ]:
# loop over the shuffled input images
for TraImagePath in TraImagePaths:

    # Read the image into a numpy array using opencv. All the read images are of different shapes
    image = cv2.imread(TraImagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize the image to be 150*150 pixels while ignoring aspect ration.
    # After reshape size of all the images will become 150*150*3. Total number of pixels in every image = 150*150*3=67500
    image = cv2.resize(image, IMAGE_SIZE)

    # Flatten converts every 3D image (150*150*3) into 1D numpy array of shape (67500,)((67500,) shape means 67500 columns and 1 row)
    image_flatten = image.flatten()

    # Append each image data 1D array to the data list
    data.append(image_flatten)

    # extract the class label from the image path and update the
    label = TraImagePath.split(os.path.sep)[-2]

    # Append each image label to the labels list
    labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
#convert the data and label list to numpy array
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
labels.shape

(300,)

In [ ]:
data.shape

(300, 67500)

In [ ]:
# import libraries
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# Split data into training and testing using 75% of the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

print("Number of training images--->",len(trainX),",","Number of training labels--->",len(trainY))
print("Number of testing images--->",len(testX),",","Number of testing labels--->",len(testY))

# convert the labels from integers to vectors by perform One hot encoding of all the labels using scikit-learn's function LabelBinarizer
# LabelBinarizer fit_transform finds all the labels
lb = preprocessing.LabelBinarizer()
# fit_transform() Learn the parameters and apply the transformation to new data
trainY = lb.fit_transform(trainY)
# transform() Apply the learned transformation to new data
testY = lb.transform(testY)

print("\n")
print ("Classes found to train",)
train_classes = lb.classes_
print(train_classes)

binary_rep_each_class = lb.transform(train_classes)
print("Binary representation of each class")
print(binary_rep_each_class)
print("\n")

Number of training images---> 225 , Number of training labels---> 225
Number of testing images---> 75 , Number of testing labels---> 75


Classes found to train
['Bed' 'Chair' 'Sofa']
Binary representation of each class
[[1 0 0]
 [0 1 0]
 [0 0 1]]




In [ ]:
# import libraries
import keras
from keras import models
from keras import layers
from keras import optimizers

import tensorflow as tf

In [ ]:
# define model using Keras
model = keras.models.Sequential()

# Our input layer has 67500 nodes, one for each of the 150 x 150 x 3 = 67500 raw pixel intensities in our flattened input images
model.add(keras.layers.Dense(1024, input_shape=(67500,), activation='relu'))
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(len(train_classes), activation="softmax"))

print ("Printing the summary of model")
model.summary()

Printing the summary of model
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 1024)              69121024  
                                                                 
 dense_13 (Dense)            (None, 512)               524800    
                                                                 
 dense_14 (Dense)            (None, 3)                 1539      
                                                                 
Total params: 69,647,363
Trainable params: 69,647,363
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Initialize # of epochs to train
EPOCHS = 9

# initialize initial learning rate
INIT_LR = 0.001

# compile the model using SGD as our optimizer and categorical
# cross-entropy loss as we have 3 classes
# opt = keras.optimizers.SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer='adam',metrics=["accuracy"])

In [ ]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY),epochs=EPOCHS, batch_size=32)

Epoch 1/9
8/8 [==============================] - 2s 53ms/step - loss: 23.4822 - accuracy: 0.5111 - val_loss: 11.9511 - val_accuracy: 0.7067
Epoch 2/9
8/8 [==============================] - 0s 25ms/step - loss: 10.5607 - accuracy: 0.5956 - val_loss: 4.0756 - val_accuracy: 0.6267
Epoch 3/9
8/8 [==============================] - 0s 31ms/step - loss: 2.5956 - accuracy: 0.6711 - val_loss: 0.4657 - val_accuracy: 0.8800
Epoch 4/9
8/8 [==============================] - 0s 32ms/step - loss: 1.6123 - accuracy: 0.7422 - val_loss: 1.6504 - val_accuracy: 0.7467
Epoch 5/9
8/8 [==============================] - 0s 29ms/step - loss: 1.2975 - accuracy: 0.7156 - val_loss: 1.8725 - val_accuracy: 0.7067
Epoch 6/9
8/8 [==============================] - 0s 24ms/step - loss: 1.1805 - accuracy: 0.7689 - val_loss: 0.4128 - val_accuracy: 0.8933
Epoch 7/9
8/8 [==============================] - 0s 24ms/step - loss: 0.2661 - accuracy: 0.9200 - val_loss: 0.3914 - val_accuracy: 0.8933
Epoch 8/9
8/8 [================

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)

# print the performance report of the prediction
print(classification_report(testY.argmax(axis=1),	predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
3/3 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

         Bed       0.78      0.90      0.84        20
       Chair       0.96      0.86      0.91        28
        Sofa       0.89      0.89      0.89        27

    accuracy                           0.88        75
   macro avg       0.88      0.88      0.88        75
weighted avg       0.89      0.88      0.88        75

